## Import important libraries

In [1]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

## Load raw data

In [2]:
raw_data = np.loadtxt('Audiobooks_data.csv', delimiter =',')
#exclude first colum(customer id) and last column(target)
inputs = raw_data[:,1:-1]
#extract the target in a separate variable
targets = raw_data[:,-1]

## Balance the dataset 

In [3]:
num_one_targets = int(np.sum(targets)) #counts the number of ones 
num_zero_targets = 0
indices_to_delete = []

#iterate over the vector of ones and zeros, 
for i in range(targets.shape[0]):
    if targets[i] == 0:
        num_zero_targets += 1
        if num_zero_targets > num_one_targets:   
            indices_to_delete.append(i)
        
input_equal_priors = np.delete(inputs,indices_to_delete, axis =0 )
targets_equal_priors = np.delete(targets, indices_to_delete, axis = 0)

#Check
if num_zero_targets - len(indices_to_delete) == num_one_targets:
    print ('Priors are equal')
else:
    print('Priors are not equal')

Priors are equal


## Standerdize the inputs

In [4]:
scaled_inputs = preprocessing.scale(input_equal_priors)
#standadization greatly improves the algorithm

## Shuffle the data

In [5]:
#shuffling makes the data more random, also improves batching
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Split data into train, validate and test datasets

In [6]:
#define the sizes of the datasets
data_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * data_count)
validation_samples_count = int(0.1 * data_count)
test_samples_count = data_count - train_samples_count - validation_samples_count

#extract the TVT datasets from the main dataset 
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

#Checking out sum of samples, no. of observations and Priors to make sure the datasets are balanced
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/ train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/ validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/ test_samples_count)

1808.0 3579 0.5051690416317407
207.0 447 0.46308724832214765
222.0 448 0.4955357142857143


## Save datasets

In [7]:
np.savez('audiobook_data_train', inputs=train_inputs, targets = train_targets   )
np.savez('audiobook_data_validation', inputs=validation_inputs, targets = validation_targets)
np.savez('audiobook_data_test', inputs=test_inputs, targets = test_targets   )